In [ ]:
!pip install -q kora 
!pip install -q keybert
!pip install -q bert-extractive-summarizer==0.10.1

     |████████████████████████████████| 57 kB 278 kB/s 
     |████████████████████████████████| 1.6 MB 2.5 MB/s 
     |████████████████████████████████| 85 kB 3.0 MB/s 
     |████████████████████████████████| 237 kB 30.8 MB/s 
     |████████████████████████████████| 51 kB 6.3 MB/s 
     |████████████████████████████████| 5.8 MB 38.0 MB/s 
     |████████████████████████████████| 1.3 MB 72.8 MB/s 
     |████████████████████████████████| 182 kB 55.1 MB/s 
     |████████████████████████████████| 7.6 MB 69.9 MB/s 


In [ ]:
from kora.selenium import wd

In [ ]:
## with stopwords
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import re
import time
from selenium import webdriver
from selenium import webdriver
from selenium.webdriver.chrome.service import Service


In [ ]:
from summarizer import Summarizer
from pprint import pprint
from summarizer import TransformerSummarizer
from keybert import KeyBERT

/usr/local/lib/python3.8/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.13) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")


In [ ]:
df = pd.DataFrame(columns = ['URL','SCRAPPED CONTENT','SUMMARY USING BERT','SUMMARY USING GPT2','TAGS Using MSD(BERT)','TAGS Using MMR (low diversity)(BERT)','TAGS Using MMR (high diversity)(BERT)','TAGS Using MSD(GPT2)','TAGS Using MMR (low diversity)(GPT2)','TAGS Using MMR (high diversity)(GPT2)'])

In [ ]:
model = Summarizer()#This is the summarizer model for bert
GPT2_model = TransformerSummarizer(transformer_type="GPT2",transformer_model_key="gpt2-medium")#This is the summarizer model for GPT2
kw_model = KeyBERT()#This is the Keyword generation model of keybert

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/350 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.2k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [ ]:
def url_classification(url):
  if not url.startswith("http"):
    url = 'http://' + url
  try:
    wd.get(url)
    time.sleep(3)
    pg=wd.page_source
  except:
    finalinfo='None'
    return finalinfo
  #driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
  #url = "https://www.bewakoof.com/"
  #driver.get(url)
  #time.sleep(3) #if you want to wait 3 seconds for the page to load
  #page_source = driver.page_source
  try:
    soup = BeautifulSoup(pg, 'html.parser')
    title = soup.find('title').text
    description = soup.find('meta', attrs={'name': 'description'})

    if "content" in str(description):
        description = description.get("content")
    else:
        description = ""

    data = ''
    tags = ['h1', 'h2', 'p', 'div', 'h3', 'a', 'h4', 'h5', 'h6', 'span']
    for i in tags:
      tag = soup.find_all(i)
      for x in range (len(tag)):
          if x == len(tag) - 1:
              data += tag[x].text
          else:
              data += tag[x].text + ". "

    allthecontent = str(data) + " " + str(description)
    allthecontent = allthecontent.lower()
    allthecontent = re.sub(r'\n+', ' ', allthecontent)
    allthecontent = re.sub('[^a-z,.]+', ' ', allthecontent)
    l = list(allthecontent.split(" "))

    finalinfo = ''
    for k in l:
      finalinfo = finalinfo + k + " "
  except:
    finalinfo='None'
    return finalinfo
  return finalinfo

In [ ]:
#link_database = pd.read_csv('Trial.csv')
link_database = pd.read_excel('divided4250.xlsx',sheet_name='Sheet5')
array=link_database.to_numpy()

In [ ]:
for i in range(len(array)):
        for j in range(len(array[i][:3])):
        #for j in range(len(array[i])):
                
                url=array[i][j]
                print(url)
                try:
                  text = url_classification(url)
                except:
                  continue
                #print(text)
                data=text[:5000]
                #print(data)

                #BERT SUMMARY
                result = model(data, num_sentences=8, min_length=80)
                full = ''.join(result)
                #pprint(full)
                sum1=full

                #GPT2 SUMMARY
                full = ''.join(GPT2_model(data, min_length=80))
                #pprint(full)
                sum2=full

                #Using Max_Sum Distance
                k11=kw_model.extract_keywords(sum1, keyphrase_ngram_range=(3, 3), stop_words='english', use_maxsum=True, nr_candidates=20, top_n=5)
                k21=kw_model.extract_keywords(sum2, keyphrase_ngram_range=(3, 3), stop_words='english', use_maxsum=True, nr_candidates=20, top_n=5)

                #Maximal Marginal Relevance (low diversity)
                k12=kw_model.extract_keywords(sum1, keyphrase_ngram_range=(3, 3), stop_words='english', use_mmr=True, diversity=0.2)
                k22=kw_model.extract_keywords(sum2, keyphrase_ngram_range=(3, 3), stop_words='english', use_mmr=True, diversity=0.2)

                #Maximal Marginal Relevance (high diversity)
                k13=kw_model.extract_keywords(sum1, keyphrase_ngram_range=(3, 3), stop_words='english', use_mmr=True, diversity=0.7)
                k23=kw_model.extract_keywords(sum2, keyphrase_ngram_range=(3, 3), stop_words='english', use_mmr=True, diversity=0.7)
                df.loc[len(df.index)] = [url,data,sum1,sum2,k11,k12,k13,k21,k22,k23] 


seu.edu.cn
stspg-customer.com
opm.gov
clipchamp.com
haufe.io
signalr.net
keybank.com
zdbb.net
privacy-center.org
klikbca.com


/usr/local/lib/python3.8/dist-packages/summarizer/cluster_features.py:149: ConvergenceWarning: Number of distinct clusters (5) found smaller than n_clusters (7). Possibly due to duplicate points in X.
  model = self._get_model(k).fit(self.features)


compuserve.com
olegb.ru
oxfam.org
pstatp.com
cdnwidget.com
clara.net
anyclip.com


/usr/local/lib/python3.8/dist-packages/summarizer/cluster_features.py:149: ConvergenceWarning: Number of distinct clusters (6) found smaller than n_clusters (7). Possibly due to duplicate points in X.
  model = self._get_model(k).fit(self.features)


avocet.io
acsalaska.net
anaconda.com
expansion.com
chop.edu
mangago.me
alexametrics.com
webcollage.net
notebookcheck.net
webopedia.com
adb.org
sparkfun.com
nudostar.com
moodys.com
aon.com
boxcloud.com
oas.org
magisto.com
qubit.com
unipd.it
queniucf.com
esm1.net
royalcaribbean.com
ssstwitter.com
oupeng.com
mediaplex.com
greatschools.org
beaxy.com
onlinehome.us
bis.org
tesla.cn


...

gfsrv.net
dvrlists.com
4kdownload.com
ionos.de
courthousenews.com
adn.com
mhanational.org
poringa.net
freeconvert.com
webmasters.com
directcrm.ru
sentry-cdn.com
tmgrup.com.tr
acompli.net
gluops.com
vivawebhost.com
db.tt
walmartimages.com
unimi.it
2046acg.com
novotelecom.ru
gadgets360.com
pandadoc.com
hostmed.ru
analdin.com
huami.com


/usr/local/lib/python3.8/dist-packages/summarizer/cluster_features.py:149: ConvergenceWarning: Number of distinct clusters (2) found smaller than n_clusters (3). Possibly due to duplicate points in X.
  model = self._get_model(k).fit(self.features)


siftscience.com
pitchbook.com
s-dc-msedge.net
i.ua
google.co.ke
lbdns-streamguys.com
labcorp.com
corsair.com
ferrari.com
nta.go.jp
bmwgroup.com


/usr/local/lib/python3.8/dist-packages/summarizer/cluster_features.py:149: ConvergenceWarning: Number of distinct clusters (3) found smaller than n_clusters (6). Possibly due to duplicate points in X.
  model = self._get_model(k).fit(self.features)


mawdoo3.com
rshb.ru
alchemer.com
0123movie.net
alphr.com
random.org
faradars.org


In [ ]:
print(df)

                    URL                                   SCRAPPED CONTENT  \
0            seu.edu.cn                                        forbidden     
1    stspg-customer.com                                               None   
2               opm.gov  featured topics. featured topics. featured top...   
3         clipchamp.com  all your video needs in one placeuse clipchamp...   
4              haufe.io  haufe l sungen f r die arbeitswelt von heute u...   
..                  ...                                                ...   
419        alchemer.com  give every customer a voice. make every voice ...   
420       0123movie.net  watch free movies online movies allows anyone ...   
421           alphr.com   if you have critical data on a word document,...   
422          random.org   www.random.org checking if the site connectio...   
423        faradars.org                                               None   

                                    SUMMARY USING BERT  \
0    

In [ ]:
df

,URL,SCRAPPED CONTENT,SUMMARY USING BERT,SUMMARY USING GPT2,TAGS Using MSD(BERT),TAGS Using MMR (low diversity)(BERT),TAGS Using MMR (high diversity)(BERT),TAGS Using MSD(GPT2),TAGS Using MMR (low diversity)(GPT2),TAGS Using MMR (high diversity)(GPT2)
0,seu.edu.cn,forbidden,,,[],[],[],[],[],[]
1,stspg-customer.com,None,,,[],[],[],[],[],[]
2,opm.gov,featured topics. featured topics. featured top...,we re hiring join the response to covid empowe...,we re hiring join the response to covid empowe...,"[(opm announced senior, 0.4674), (staffing ali...","[(opm human resources, 0.6606), (federal workf...","[(opm human resources, 0.6606), (executive ord...","[(lead serve federal, 0.4211), (government ent...","[(serve federal government, 0.5521), (workforc...","[(serve federal government, 0.5521), (response..."
3,clipchamp.com,all your video needs in one placeuse clipchamp...,all your video needs in one placeuse clipchamp...,all your video needs in one placeuse clipchamp...,"[(clipchamp aspiring creator, 0.5685), (record...","[(video templates best, 0.7093), (wedding vide...","[(video templates best, 0.7093), (editor need ...","[(beautiful videos today, 0.3879), (join commu...","[(clipchamp free video, 0.7272), (video clipch...","[(clipchamp free video, 0.7272), (creator seas..."
4,haufe.io,haufe l sungen f r die arbeitswelt von heute u...,haufe l sungen f r die arbeitswelt von heute u...,haufe l sungen f r die arbeitswelt von heute u...,"[(der hr funktion, 0.4014), (das personalmagaz...","[(digitalisierung der hr, 0.5793), (die neue a...","[(digitalisierung der hr, 0.5793), (das person...","[(digitalen transformation ob, 0.4304), (chste...","[(steuerberatung immobilienwirtschaft oder, 0....","[(steuerberatung immobilienwirtschaft oder, 0...."
...,...,...,...,...,...,...,...,...,...,...
419,alchemer.com,give every customer a voice. make every voice ...,make every voice matter.become customer centri...,make every voice matter.become customer centri...,"[(cx automation flexible, 0.413), (voice custo...","[(customer centric alchemer, 0.6195), (custome...","[(customer centric alchemer, 0.6195), (cx auto...","[(collect customer data, 0.3865), (transformin...","[(survey alchemer takes, 0.684), (data survey ...","[(survey alchemer takes, 0.684), (cx automatio..."
420,0123movie.net,watch free movies online movies allows anyone ...,watch free movies online movies allows anyone ...,watch free movies online movies allows anyone ...,"[(subscribe movies mailing, 0.477), (statesaus...","[(free movies online, 0.6863), (movies online ...","[(free movies online, 0.6863), (advertisements...","[(allows watch online, 0.4414), (subscribe mov...","[(free movies online, 0.6949), (online movies ...","[(free movies online, 0.6949), (advertisements..."
421,alphr.com,"if you have critical data on a word document,...","if you have critical data on a word document, ...","if you have critical data on a word document, ...","[(mechanism keeps away, 0.1694), (away prying ...","[(document help protection, 0.6693), (word doc...","[(document help protection, 0.6693), (data wor...","[(mechanism keeps away, 0.1694), (away prying ...","[(document help protection, 0.6693), (word doc...","[(document help protection, 0.6693), (data wor..."
422,random.org,www.random.org checking if the site connectio...,www.random.org checking if the site connection...,www.random.org checking if the site connection...,"[(ensures connection secure, 0.452), (seeing p...","[(page requests malicious, 0.6405), (site ensu...","[(page requests malicious, 0.6405), (connectio...","[(secure enable javascript, 0.3961), (occasion...","[(connection secure www, 0.6269), (site ensure...","[(connection secure www, 0.6269), (www random ..."


In [ ]:
df.to_excel('scrapped.xlsx')